In [1]:
#importing first
import urllib.request, urllib.parse, urllib.error
from bs4 import BeautifulSoup
import ssl
import sqlite3
# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE
#getting url to scrap
url = input('Enter - ')
#parsing start
req = urllib.request.Request(url, headers={'User-Agent': 'Mozilla/5.0'})
html = urllib.request.urlopen(req).read()
soup = BeautifulSoup(html, 'html.parser')

Enter - https://www.ncs.gov.in/Pages/default.aspx


In [3]:
sectors=soup.find_all("li",class_="jobVacanciesSectorOption")

In [40]:
#Making a list of all jobs available
#states=soup.find_all("li",class_="jobVacanciesOption")
sector_vacancy=[]
for sector in sectors:
    link=sector.find('a').get('href',None)
    name=sector.find('span',class_="sectorName").string
    description=sector.find('span',class_="sectorName").get('title',None)
    vacancy=sector.find('span',class_="sectorJobsPost").string
    result=[link,name,description,vacancy]
    sector_vacancy.append(result)

In [43]:
sector_vacancy[0][1]

'Agriculture and Related Activities'

In [47]:
p=[]
for i in range(len(sector_vacancy)):
    l=[]
    link=sector_vacancy[i][0]
    req = urllib.request.Request(link, headers={'User-Agent': 'Mozilla/5.0'})
    html = urllib.request.urlopen(req).read()
    soup=BeautifulSoup(html,'html.parser')
    vacancies=soup.find_all(id="mytab")
    for vacancy in vacancies:
        try:
            form_page=vacancy.find('div').find('a').get('onclick')[14:-2]
            req = urllib.request.Request(form_page, headers={'User-Agent': 'Mozilla/5.0'})
            html = urllib.request.urlopen(req).read()
            soup=BeautifulSoup(html,'html.parser')
            last_date=re.findall('Last .+?2020',soup.text.replace('\n',"").replace("  ","").replace("\r",""))[0][18:]
            last_date="-".join(last_date.split('/')[::-1])
            Post=vacancy.find('div').find_all('span')[0].string
            Company_Name=vacancy.find('div').find_all('span')[4].string+vacancy.find('div').find_all('span')[5].string
            Location=vacancy.find('div').find_all('span')[8].string
            Expected_Salary=vacancy.find('div').find_all('span')[11].string
            Skill_required=vacancy.find('div').find_all('span')[14].string.replace("\t"," ").replace("  "," ")
            Description=vacancy.find('div').find_all('span')[16].string
            if Description!=None:
                Description=Description.replace("\xa0","")
#             with sqlite3.connect("MainDatabase.sqlite") as conn:
#                 cur=conn.cursor()
#                 cur.execute('''SELECT Job_name,Skill_required,Company_Name FROM Jobs Where Job_name=? AND Company_Name=?''',(Post,Company_Name))
#                 row=cur.fetchone()
#                 if row!=None:
#                     continue
#             print("Found")
            r=[form_page,Post,Company_Name,Location,Expected_Salary,Skill_required,Description,sector_vacancy[i][1]]
#             with sqlite3.connect("MainDatabase.sqlite") as conn:
#                 cur=conn.cursor()
#                 cur.execute('''INSERT OR IGNORE INTO Jobs(Job_name,Skill_required,Job_Description,Expected_salary,state_Name,Location,Company_Name,link,last_date)
#                 VALUES(?,?,?,?,?,?,?,?,?)''',(Post,Skill_required,Description,Expected_Salary,state_vacancy[i][1],Location,Company_Name,form_page,last_date))
            l.append(r)
#             conn.commit()
        except Exception as ex:
            print(ex,i)
    p.append(l)
            

'NoneType' object has no attribute 'replace' 12
<urlopen error [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond> 12
<urlopen error [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond> 17
<urlopen error [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond> 20


In [74]:
for i in sector_vacancy:
    print(i[1])

Agriculture and Related Activities
Arts and Entertainment
Civil and Construction Works
Education
Finance and Insurance
Health
Hotels, Food Service and Catering
Household and Domestic Work
International Organizations
IT and Communication
Manufacturing
Mining And Quarrying
Operations and Support
Other Service Activities
Power and Energy
Public Administration and Defense
Real Estate Activities
Specialized Professional Services
Transportation and Storage
Water Supply, Sewerage and Waste Management
Wholesale and Retail
Sector Not Specified


In [56]:
l

[['https://www.ncs.gov.in/Pages/ViewJobDetails.aspx?A=w1BcJXzB%2BW4%3D&U=&JSID=Cn0Jg8DqlpI%3D&RowId=Cn0Jg8DqlpI%3D',
  'electrical',
  'SIDDHI VINAYAK CONSTRUCTION\r\n                                            Job Location:',
  '\r\n                                            Salary:',
  '\r\n                                            Skill Required: ',
  '\r\n                      Job Description:',
  None,
  'Sector Not Specified'],
 ['https://www.ncs.gov.in/Pages/ViewJobDetails.aspx?A=w1BcJXzB%2BW4%3D&U=&JSID=Euxc18DErSc%3D&RowId=Euxc18DErSc%3D',
  'Sales Executives',
  'MAANI CARE SYSTEM INDIA PRIVATE LIMITED\r\n                                            Job Location:',
  '\r\n                                            Salary:',
  '\r\n                                            Skill Required: ',
  '\r\n                      Job Description:',
  None,
  'Sector Not Specified'],
 ['https://www.ncs.gov.in/Pages/ViewJobDetails.aspx?A=w1BcJXzB%2BW4%3D&U=&JSID=cUCFqwXD%2BDs%3D&RowI

In [25]:
try:
    with sqlite3.connect("MainDatabase.sqlite") as conn:  
        cur=conn.cursor()
        cur.execute('''SELECT link,Job_id FROM Jobs''')
        row=cur.fetchall()
        print(len(row))
        for i in range(len(row)):
            link=row[i][0]
            job_id=row[i][1]
            if link=='NULL':
                continue
            req = urllib.request.Request(link, headers={'User-Agent': 'Mozilla/5.0'})
            html = urllib.request.urlopen(req).read()
            soup=BeautifulSoup(html,'html.parser')
            sector=soup.find_all('div',class_="row")[8].find_all('label')[3].string.replace('  ',"")[2:-2]
            minimum_qualification=soup.find_all('div',class_="row")[11].find_all('label')[1].string.replace("  ","")[2:-2]
            last_date=re.findall('Last date to apply.+?2020',soup.text.replace('\n',"").replace("  ","").replace("\r",""))[0][18:]
            cur.execute('''UPDATE Jobs SET minimum_qualification=? WHERE Job_id=?''',(minimum_qualification,job_id))
            print(i,"done")
            conn.commit()
except Exception as e:
    print(e)
    pass

555
0 done
1 done
2 done
3 done
4 done
5 done
6 done
7 done
8 done
9 done
10 done
11 done
12 done
13 done
14 done
15 done
16 done
17 done
18 done
19 done
20 done
21 done
22 done
23 done
24 done
25 done
26 done
27 done
28 done
29 done
30 done
31 done
32 done
33 done
34 done
35 done
36 done
37 done
38 done
39 done
40 done
41 done
42 done
43 done
44 done
45 done
46 done
47 done
48 done
49 done
50 done
51 done
52 done
53 done
54 done
55 done
56 done
57 done
58 done
59 done
60 done
61 done
62 done
63 done
64 done
65 done
66 done
67 done
68 done
69 done
70 done
71 done
72 done
73 done
74 done
75 done
76 done
77 done
78 done
79 done
80 done
81 done
82 done
83 done
84 done
85 done
86 done
87 done
88 done
89 done
90 done
91 done
92 done
93 done
94 done
95 done
96 done
97 done
98 done
99 done
100 done
101 done
102 done
103 done
104 done
105 done
106 done
107 done
108 done
109 done
110 done
111 done
112 done
113 done
114 done
115 done
116 done
117 done
118 done
119 done
120 done
121 done
122 done

In [7]:
import urllib.request, urllib.parse, urllib.error
from bs4 import BeautifulSoup
import ssl
import sqlite3
import re

In [9]:
form_page='https://www.ncs.gov.in/Pages/ViewJobDetails.aspx?A=w1BcJXzB%2BW4%3D&U=&JSID=CKvaAeUuSzI%3D&RowId=CKvaAeUuSzI%3D'
req = urllib.request.Request(form_page, headers={'User-Agent': 'Mozilla/5.0'})
html = urllib.request.urlopen(req).read()
soup=BeautifulSoup(html,'html.parser')
# last_date=re.findall('Last .+?2020',soup.text.replace('\n',"").replace("  ","").replace("\r",""))[0][18:]
# last_date="-".join(last_date.split('/')[::-1])
# Post=vacancy.find('div').find_all('span')[0].string
# Company_Name=vacancy.find('div').find_all('span')[4].string+vacancy.find('div').find_all('span')[5].string
# Location=vacancy.find('div').find_all('span')[8].string
# Expected_Salary=vacancy.find('div').find_all('span')[11].string
# Skill_required=vacancy.find('div').find_all('span')[14].string.replace("\t"," ").replace("  "," ")
# Description=vacancy.find('div').find_all('span')[16].string
# if Description!=None:
# Description=Description.replace("\xa0","")

In [24]:
minimum_qualification=soup.find_all('div',class_="row")[11].find_all('label')[1].string.replace("  ","")[2:-2]

'Post Graduate'

In [26]:
def free_jobs():
    with sqlite3.connect("MainDatabase.sqlite") as conn:
        cur=conn.cursor()
        cur.execute('''SELECT * FROM Jobs WHERE Recruiter_id IS NOT NULL''')
        row=cur.fetchall()
        return row

In [35]:
type(free_jobs()[1][0])

int

In [18]:
import sqlite3
def recruiter_job(id):
    with sqlite3.connect("MainDatabase.sqlite") as conn:
        cur=conn.cursor()
        cur.execute('''SELECT User_id,Recruiter_id from Applications WHERE Job_id=?''',(id,))
        rows=cur.fetchall()
        if len(rows)!=0:
            for row in range(len(rows)):
                user_id=rows[row][0]
                cur.execute('''SELECT Name,Phone,Email,Address,Marks_10,Marks_12,DOB,Gender,Category FROM Users WHERE User_id=?''',(user_id,))
                rows[row]=rows[row]+cur.fetchone()
        return rows

In [19]:
recruiter_job(1)

OperationalError: no such table: Users

In [11]:
with sqlite3.connect("MainDatabase.sqlite") as conn:
    cur=conn.cursor()
    cur.execute('''INSERT INTO Applications(User_id,Recruiter_id,Job_id) VALUES(?,?,?)''',(1,1,1))
    row=cur.fetchone()
    conn.commit()

In [15]:
rows

NameError: name 'rows' is not defined

In [52]:
(1,)+(2,)

(1, 2)

In [6]:
with sqlite3.connect("MainDatabase.sqlite") as conn:
    cur=conn.cursor()
    cur.execute('''
    CREATE TABLE IF NOT EXISTS Applications (
        Application_id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
        User_id INTEGER,
        Recruiter_id INTEGER,
        Job_id INTEGER,
        FOREIGN KEY (User_id)
        REFERENCES Users (User_id),
        FOREIGN KEY (Recruiter_id)
        REFERENCES Recruiters (Recruiter_id),
        FOREIGN KEY (Job_id)
        REFERENCES Jobs (Job_id)
    );
    ''')
    conn.commit()

In [1]:
def job_apply(User_id,Job_id):
    with sqlite3.connect("MainDatabase.sqlite") as conn:
        cur=conn.cursor()
        cur.execute('''SELECT Recruiter_id FROM Jobs WHERE Job_Id=?''',(Job_id,))
        Recruiter_id=cur.fetchone()[0]
        cur.execute('''INSERT INTO Applications User_id,Recruiter_id,Job_id VALUES(?,?
        )''',(User_id,Recruiter_id,Job_id))
        conn.commit
        return 1

In [7]:
with sqlite3.connect("MainDatabase.sqlite") as conn:
    cur=conn.cursor()
    cur.execute('''SELECT Recruiter_id FROM Jobs WHERE Job_Id=?''',(Job_id,))
    Recruiter_id=cur.fetchone()[0]
#     cur.execute('''INSERT INTO Applications User_id,Recruiter_id,Job_id VALUES(?,?
#     )''',(User_id,Recruiter_id,Job_id))
    conn.commit

OperationalError: no such table: Jobs

In [4]:
import sqlite3

In [7]:
from json import loads

In [5]:
a='["2","3","4"]'

In [ ]:
with sqlite3.connect("MainDatabase.sqlite"):

In [9]:
import sqlite3

In [ ]:
def fetch_recommend(State_Name,sector):
    with sqlite3.connect("MainDatabase.sqlite") as conn:
        cur=conn.cursor()
        cur.execute('''SELECT Job_name,Skill_required,Job_Description,Expected_salary,State_Name,Location,Company_Name,sector,last_date,minimum_qualification FROM Jobs WHERE State_Name=? AND sector=?''',(State_Name,sector))
        row=cur.fetchall()
        return row